In [10]:
import pandas as pd
import numpy as np

In [11]:
# get data
df = pd.read_excel(r'S:\L2S\CODE\PRIVATE\ecovio_trial\sealability_dataset.xlsx', sheet_name='verf_dataset')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   temperature         100 non-null    int64  
 1   holding_time_inSec  100 non-null    float64
 2   lid_coating_gsm     100 non-null    int64  
 3   cup_pair_inOz       100 non-null    int64  
 4   cup_coating_gsm     100 non-null    int64  
 5   status (ok/not)     100 non-null    object 
 6   percentage_cov      100 non-null    float64
dtypes: float64(2), int64(4), object(1)
memory usage: 5.6+ KB


In [12]:
df = df.rename(columns={'status (ok/not)': 'result'})
df.describe()

,temperature,holding_time_inSec,lid_coating_gsm,cup_pair_inOz,cup_coating_gsm,percentage_cov
count,100.000000,100.000000,100.000000,100.000000,100.0,100.000000
mean,326.300000,1.740000,23.140000,14.560000,30.0,0.825000
std,173.039699,0.690776,5.672938,1.929673,0.0,0.226134
min,110.000000,1.000000,18.000000,12.000000,30.0,0.250000
25%,177.500000,1.000000,18.000000,12.000000,30.0,0.750000
50%,310.000000,1.500000,20.000000,16.000000,30.0,1.000000
75%,420.000000,2.000000,30.000000,16.000000,30.0,1.000000
max,700.000000,3.000000,30.000000,16.000000,30.0,1.000000


**Data Description**

This is to understand each column for initial analysis

|Column|Description|
|---|---|
|temperature|temperature, set during sealing process|
|holding_time_inSec|dwelling time, how long the sealing process is performed|
|lid_coating_gsm|how thick the paper, after the extrusion process|
|cup_pair_inOz|what size the cup used to be paired with heat seal lid (this might not be important to be trained)|
|cup_coating_gsm|the coating thickness of paper cup, this is good parameter if it has various range|
|result|this is a label|
|percentage_cov|pretty important to check anomaly between cups, lids, and sealers|

In [13]:
# select features and target
target = ['result']
X = df.drop(columns=target)
y = df[target]

**Train Test Split**

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print('Train set:', X_train.shape, y_train.shape)
print('Test set:', X_test.shape, y_test.shape)

Train set: (80, 6) (80, 1)
Test set: (20, 6) (20, 1)


## Support Vector Machine

Utilizing this model, I'll play around with one of 4 kernels (linear, polynomial, radial basis function (rbf), and sigmoid). But I'll go over 4 types later on with GridSearch to get the best one.

In [20]:
from sklearn.svm import SVC
from sklearn import svm

model_svm = svm.SVC(kernel='poly', gamma=0.0009) # define
model_svm.fit(X_train, y_train) # fit/train
model_svm.score(X_test, y_test) # evaluate

C:\Users\Pongo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.95

**Cross Validation**

In [21]:
from sklearn.model_selection import GridSearchCV

# define
grid_svm = SVC()
params_svm = {
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'gamma': range(int(0.1 * 100), int(0.001 * 100) + 1, -1),
}
grid_svm = GridSearchCV(grid_svm, param_grid=params_svm, cv=10)

# fit/train
grid_svm.fit(X_train, y_train)

# predict
print(grid_svm.score(X_test, y_test))

# find the best parameter
print(grid_svm.best_params_)

C:\Users\Pongo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Pongo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Pongo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the sha

1.0
{'gamma': 10, 'kernel': 'poly'}


C:\Users\Pongo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Pongo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Pongo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the sha

**Predict Value**

In [22]:
y_pred_svm = model_svm.predict(X_test)
print(y_pred_svm[0:5])
pd.DataFrame(X_test[0:5])

['not' 'ok' 'not' 'ok' 'not']


,temperature,holding_time_inSec,lid_coating_gsm,cup_pair_inOz,cup_coating_gsm,percentage_cov
50,650,1.0,30,16,30,0.75
89,610,1.0,20,12,30,1.00
72,370,1.0,30,16,30,0.25
3,145,1.0,18,12,30,1.00
98,280,1.5,20,12,30,0.50


## K-Nearest Neighbor

This model will explore assorted metrics such as manhattan, euclidean, minkowski, hamming, cosine, jaccard, and chebyshev

In [23]:
from sklearn.neighbors import KNeighborsClassifier

model_knn = KNeighborsClassifier(n_neighbors = 1, metric = 'hamming')
model_knn.fit(X_train, y_train)
model_knn.score(X_test, y_test)

C:\Users\Pongo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neighbors\_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


1.0

**Cross Validation**

In [24]:
grid_knn = KNeighborsClassifier()
params_knn = {
    'n_neighbors': range(1,9),
    'metric': ['euclidean', ' minkowski', 'manhattan', 'hamming', 'cosine', 'jaccard', 'chebyshev']
}
grid_knn = GridSearchCV(grid_knn, param_grid=params_knn, cv=10)

grid_knn.fit(X_train, y_train) # train
print(grid_knn.score(X_test, y_test))

# find the best parameter
print(grid_knn.best_params_)

C:\Users\Pongo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neighbors\_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\Pongo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neighbors\_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
C:\Users\Pongo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neighbors\_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change t

1.0
{'metric': 'hamming', 'n_neighbors': 1}


C:\Users\Pongo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\model_selection\_search.py:1103: UserWarning: One or more of the test scores are non-finite: [0.9    0.8875 0.8625 0.775  0.7625 0.775  0.75   0.75      nan    nan
    nan    nan    nan    nan    nan    nan 0.9    0.8875 0.8625 0.775
 0.7875 0.775  0.75   0.7625 0.975  0.9625 0.95   0.95   0.9    0.875
 0.875  0.85   0.9    0.875  0.8625 0.775  0.8    0.775  0.7125 0.7875
 0.575  0.55   0.55   0.55   0.575  0.575  0.575  0.575  0.8875 0.875
 0.8625 0.775  0.7125 0.725  0.725  0.75  ]
  warnings.warn(
C:\Users\Pongo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neighbors\_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return se

**Predict**

In [25]:
y_pred_knn = model_knn.predict(X_test)
print(y_pred_knn[0:5])
pd.DataFrame(X_test[0:5])

['not' 'ok' 'not' 'ok' 'not']


,temperature,holding_time_inSec,lid_coating_gsm,cup_pair_inOz,cup_coating_gsm,percentage_cov
50,650,1.0,30,16,30,0.75
89,610,1.0,20,12,30,1.00
72,370,1.0,30,16,30,0.25
3,145,1.0,18,12,30,1.00
98,280,1.5,20,12,30,0.50


## Neural Network

explore: tanh, sigmoid, and relu

In [26]:
from sklearn.neural_network import MLPClassifier

model_mlp = MLPClassifier(hidden_layer_sizes=(50, 25, 10), activation = 'tanh', max_iter = 10000)
model_mlp.fit(X_train, y_train)
model_mlp.score(X_test, y_test)

C:\Users\Pongo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1105: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.95

**Cross Validation**

In [27]:
grid_mlp = MLPClassifier()
params_mlp = {
    'activation': ['tanh', 'sigmoid', 'relu']
}
grid_mlp = GridSearchCV(grid_mlp, param_grid=params_mlp, cv=10)

grid_mlp.fit(X_train, y_train) # train
print(grid_mlp.score(X_test, y_test))

# find the best parameter
print(grid_mlp.best_params_)

C:\Users\Pongo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1105: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Pongo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Pongo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1105: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change

0.55
{'activation': 'tanh'}


C:\Users\Pongo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Pongo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1105: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\Pongo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization 

## Evaluation

In [28]:
y_svm = model_svm.predict(X_test)
y_knn = model_knn.predict(X_test)
y_mlp = model_mlp.predict(X_test)

**Confusion Matrix**

In [40]:
from sklearn.metrics import confusion_matrix
(confusion_matrix(y_test, y_svm)) # support vector machine

array([[ 8,  1],
       [ 0, 11]], dtype=int64)

In [33]:
confusion_matrix(y_test, y_knn) # k-nearest neighbor

array([[ 9,  0],
       [ 0, 11]], dtype=int64)

In [32]:
confusion_matrix(y_test, y_mlp) # nerual network

array([[ 8,  1],
       [ 0, 11]], dtype=int64)